In [0]:
dbutils.widgets.text("p_file_date",'',"Enter the date in yyyymmdd")
v_file_date = dbutils.widgets.get("p_file_date")
v_file_date

Out[11]: '20241223'

In [0]:
import os

if os.path.isfile(f'/dbfs/data/landing/sales_data_{v_file_date}.csv'):
     print("File Exists") 
else:            
     print("File doesn't exists")
     raise Exception("File doesn't exists")

File doesn't exists


---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
<command-2274317134576382> in <cell line: 3>()
      5 else:
      6      print("File doesn't exists")
----> 7      raise Exception("File doesn't exists")

Exception: File doesn't exists

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [0]:
csvSchema = StructType(fields = [  StructField("transaction_id", IntegerType(), False)
                                      , StructField("customer_id", IntegerType(), False)
                                      , StructField("product_id", IntegerType(), False)
                                      , StructField("quantity", IntegerType(), True)
                                      , StructField("total_price", DoubleType(), True)
                                      , StructField("sales_date", StringType(), True)
                                     ]
                            )
      

In [0]:
df_csv = (spark
          .read.schema(csvSchema).options(header='true').csv(f'/data/landing/sales_data_{v_file_date}*.csv')
         )
         


In [0]:
df_csv.createOrReplaceTempView("sales_data_csv");


In [0]:
df_cleaned = spark.sql("""select transaction_id
                           , customer_id
                           , product_id
                           , coalesce(quantity,0) quantity
                           , coalesce(total_price,0) as total_price
                           , to_timestamp(sales_date) as sales_date 
                      from sales_data_csv                      
                     where transaction_id is not null
                     or    customer_id is not null
                     or    product_id is not null""");

In [0]:
df_cleaned.write.format("parquet").mode("append").save("/data/staging")